In [97]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from urllib.request import urlopen, Request
from selenium.webdriver.support import expected_conditions as EC

In [98]:
driver = webdriver.Chrome()
url = 'https://th.investing.com/earnings-calendar/'
driver.get(url)

# Try to close popup if it appears
try:
    popup_close = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".popupCloseIcon.largeBannerCloser"))
    )
    popup_close.click()
    print("Found and clicked the 'Close' button!")
except Exception as e:
    print("Popup close button not found or not clickable:", e)

# Click the Filters button
try:
    filters_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "filterStateAnchor"))
    )
    filters_button.click()
    print("Found and clicked the 'Filters' button!")
except Exception as e:
    print("Filters button not found or not clickable:", e)

# Click the Clear button
try:
    erase_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "ล้างข้อมูล"))
    )
    erase_button.click()
    print("Found and clicked the 'Clear' button!")
except Exception as e:
    print("Clear button not found or not clickable:", e)

# Click the United States button
try:
    country_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "country5"))
    )
    country_button.click()
    print("Found and clicked the 'United States' button!")
except Exception as e:
    print("United States button not found or not clickable:", e)

# Click the 2 Star button
try:
    star2_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "importance2"))
    )
    star2_button.click()
    print("Found and clicked the '2 Star' button!")
except Exception as e:
    print("2 Star button not found or not clickable:", e)

# Click the 3 Star button
try:
    star3_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "importance3"))
    )
    star3_button.click()
    print("Found and clicked the '3 Star' button!")
except Exception as e:
    print("3 Star button not found or not clickable:", e)

# Click the Apply button
try:
    apply_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "ecSubmitButton"))
    )
    apply_button.click()
    print("Found and clicked the 'Apply' button!")
except Exception as e:
    print("Apply button not found or not clickable:", e)


time.sleep(5)   
response = driver.page_source
soup = BeautifulSoup(response, 'html.parser')
table = soup.find_all("tbody")
driver.quit()

Popup close button not found or not clickable: Message: 

Found and clicked the 'Filters' button!
Found and clicked the 'Clear' button!
Found and clicked the 'United States' button!
Found and clicked the '2 Star' button!
Found and clicked the '3 Star' button!
Found and clicked the 'Apply' button!


In [101]:
def earnings_calendar(table_bodies):
    data = []
    current_date = None

    for tbody in table_bodies:
        for tr in tbody.find_all("tr"):
            date_td = tr.find(class_="theDay")
            if date_td:
                current_date = date_td.text.strip()
            else:
                company_span = tr.find(class_="earnCalCompanyName middle")
                ticker_a = tr.find(class_="bold middle")
                all_left_strong_tds = tr.find_all(class_="leftStrong")
                market_cap = tr.find(class_="right")
                if tr.find(class_="marketOpen genToolTip oneliner reverseToolTip"):
                    outtime_status = "Before Market"
                elif tr.find(class_="marketClosed genToolTip oneliner reverseToolTip"):
                    outtime_status = "After Market"
                else:
                    outtime_status = "N/A"
                estimate_eps = 'N/A'
                estimate_revenue = 'N/A'
                
                if company_span and ticker_a:
                    company = company_span.text.strip() if company_span else None
                    ticker = ticker_a.text.strip() if ticker_a else None
                    estimate_eps = all_left_strong_tds[0].text.replace('/','').strip() if estimate_eps else None
                    estimate_revenue = all_left_strong_tds[1].text.replace('/','').strip() if estimate_revenue else None
                    market_cap = market_cap.text.strip() if market_cap else None
                    
                    date=datetime.strptime(current_date, '%A, %B %d, %Y')
                    
                    data.append({
                        'date': date,
                        'company': company,
                        'ticker': ticker,
                        'estimate_eps': estimate_eps,
                        'estimate_revenue': estimate_revenue,
                        'market_cap': market_cap,
                        'outtime': outtime_status
                    })
                else:
                    continue
    return data

In [102]:
earnings_calendar_data = earnings_calendar(table)
df = pd.DataFrame(earnings_calendar_data)

In [103]:
df.head(20)

,date,company,ticker,estimate_eps,estimate_revenue,market_cap,outtime
0,2025-05-20,Home Depot,HD,3.59,39.25B,377.08B,Before Market
1,2025-05-20,Palo Alto Networks,PANW,0.771,2.28B,128.65B,After Market
2,2025-05-20,Keysight Technologies,KEYS,1.65,1.28B,28.13B,After Market
3,2025-05-21,TJX,TJX,0.9124,13B,150.76B,Before Market
4,2025-05-21,Lowe's,LOW,2.91,21.03B,131.21B,Before Market
5,2025-05-21,Medtronic,MDT,1.58,8.82B,111.54B,Before Market
6,2025-05-21,Target,TGT,1.73,24.46B,44.52B,Before Market
7,2025-05-21,Best Buy,BBY,1.07,8.75B,15.16B,N/A
8,2025-05-22,Intuit,INTU,10.9,7.56B,187.55B,After Market
9,2025-05-22,Analog Devices,ADI,1.69,2.51B,111.37B,Before Market


In [104]:
now = datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
df.to_csv(f'Data/Earnings_Calendar/{date_time}.csv', index=False)